In [1]:
import os, sys
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW
reload(RTCW)

db_name = 'qcon03.db'
demos_folder = 'demosqcon03'
cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [4]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
demo_df = db.get_table_as_df(table_name= 'demo')
player_df = db.get_table_as_df(table_name= 'player')
chatmessages_df = db.get_table_as_df(table_name= 'chatmessage')
chatmessages_df = RTCW.add_match_data(chatmessages_df, player_df, demos_dct, what_df = 'chatmessages_df')
chatmessages_df = RTCW.feature_extraction_chat(chatmessages_df)

obituary table fetched in 1.04 seconds
demo table fetched in 0.11 seconds
player table fetched in 0.15 seconds
chatmessage table fetched in 0.84 seconds


In [9]:
def get_docruns(chatmessages_df, min_docrun_length = None, max_timeleft = None, docs_succesful = None, min_docs_lost = None, verbose = True):
    
    #empty lists that we will populate and use to make final df
    pd_md5 = []
    pd_start_docrun_dwtime = []
    pd_end_docrun_dwtime = []
    pd_start_docrun_secsleft = []
    pd_end_docrun_secsleft = []
    pd_docrun_duration = []
    pd_times_docs_lost = []
    pd_won_round = []
    pd_demo_name = []
    pd_match_name = []
    
    #helper variables for verbose
    counter = 0
    total_demos = obituary_df.szMd5.nunique()
    
    for demo in chatmessages_df.szMd5.unique():
        #evt hier nog cutten op timelimit hit of docsevents zoals cell hierboven voor speed
        df_demo = chatmessages_df.loc[(chatmessages_df['szMd5'] == demo) & (chatmessages_df['InMatch'])]
        demo_name = df_demo.demoName.unique()
        match_name = df_demo.matchName.unique()
        counter += 1
        
        arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])
        taken_docs_bool = False
        docs_lost_times = 0
        
        for row in range(len(arr)):
            chat = arr[row, :]
            
            #taken docs
            if chat[0] == 1:
                if taken_docs_bool == False: 
                    start_dwtime = chat[2]
                    start_timeleft = chat[3]
                    taken_docs_bool = True
                else:
                    docs_lost_times += 1

            #transmitted docs
            if chat[0] == 2:
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool = False
                docs_succes = True
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
            
                docs_lost_times = 0
                
            #returned docs / timelimit hit
            if ((chat[0] == 0 or chat[1] == 1) and (taken_docs_bool == True)):
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool = False
                docs_succes = False
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
                
                docs_lost_times = 0
                
            #if demo stopped before end of docrun
            
            
        #verbose shizzle
        if counter % 100 == 0:
            if verbose:
                print 'scanned ' + str(counter) + ' demos of ' + str(total_demos) + ' demos in total' 
    
    print 'all done!'
    
    
    #make final dataframe where 1 row is a spree with all the necessary info
    df_docs = pd.DataFrame(
    {'md5': pd_md5,
     'start_dwtime': pd_start_docrun_dwtime,
     'end_dwtime': pd_end_docrun_dwtime,
     'start_secsleft': pd_start_docrun_secsleft,
     'end_secsleft': pd_end_docrun_secsleft,
     'duration': pd_docrun_duration,
     'times_lost_docs': pd_times_docs_lost,
     'won_round': pd_won_round,
     'demo': pd_demo_name,
     'match': pd_match_name,
    })
    
    
    
    if min_docrun_length != None:
        df_docs = df_docs.loc[df_docs['duration'] >= min_docrun_length]
        
    if max_timeleft != None:
        df_docs = df_docs.loc[df_docs['end_secsleft'] <= max_timeleft]
        
    if docs_succesful != None:
        df_docs = df_docs.loc[df_docs['won_round'] == docs_succesful]
        
    if min_docs_lost != None:
        df_docs = df_docs.loc[df_docs['times_lost_docs'] >= min_docs_lost]
        
    return df_docs    

In [10]:
reload(RTCW)
res = RTCW.get_docruns(chatmessages_df, 
                       min_docrun_length = 10, 
                       max_timeleft = 999, 
                       docs_succesful = True, 
                       min_docs_lost = 2, 
                       verbose = True)
res[['start', 'end', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round', 'md5']]

C:\Users\Jelle\Documents\GIT\rtcw_demo_analyzer\src\rtcw_functions.py:733: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])


scanned 100 demos of 726 demos in total
scanned 200 demos of 726 demos in total
scanned 300 demos of 726 demos in total
scanned 400 demos of 726 demos in total
scanned 500 demos of 726 demos in total
scanned 600 demos of 726 demos in total
scanned 700 demos of 726 demos in total
all done!


,start,end,start_secsleft,end_secsleft,duration,times_lost_docs,won_round,md5
38,17287250,17375400,306,217,89,2,True,b6c782719282e38958a650b4e8bd911c
44,17287250,17375400,306,217,89,2,True,dce31aac33c7915f444360b3a15f6ddd
50,17287250,17375400,306,217,89,2,True,b2aa2417b76e07351832297ec8215da5
56,17287250,17375400,306,217,89,2,True,263d683cefd2a14c705702d3c55ad718
62,17287250,17375400,306,217,89,2,True,16adac540d8cb0689ea34144ba79c2c3
110,17287250,17375400,306,217,89,2,True,1e3f59c6d9ca4d733b8b4e5ab314b15d
374,7114250,7146450,194,161,33,2,True,a11f104bb904862dd2e56e5bc4fbdbca
617,6466250,6557100,477,387,90,2,True,ec2fd1df7ffdc2a51e37365cb338f9b0
619,6466250,6557100,477,387,90,2,True,3f563e27b82d142cfcfb375399fb7587
621,6466250,6557100,477,387,90,2,True,de2002c0da56eae3c19ef2760d48499f


In [12]:
res = res[res['times_lost_docs'] == 4]

In [13]:
reload(RTCW)
RTCW.cut_demos(root_path, demos_dct, res, demo_type = 'docs', offset_start = 8, offset_end = 8, transform_to_dm_60 = True, demo_folder_name= 'demos')

In [15]:
reload(RTCW)
RTCW.generate_capture_list(res, demo_type='docs', transform_to_dm_60 = True)

In [ ]:
spree = res.iloc[0]
spree['demo'][:-6] + '_' + str(spree['duration']) + '_' + str(spree['start_secsleft']) + '_' + str(spree['end_secsleft']) + '_' + str(spree['won_round']) + '_' + str(spree['start_dwtime']) + spree['demo'][-6:] 